In [1]:
import numpy as np

def custom_svd(matrix):
    """
    Performs a custom Singular Value Decomposition (SVD) on a given matrix and computes its inverse, 
    condition number, and reconstructs the original matrix.

    Parameters:
    ----------
    matrix : np.ndarray
        A 2D NumPy array representing the matrix to decompose.

    Returns:
    -------
    U : np.ndarray
        Left singular vectors (U matrix) of the input matrix.
    S_diag : np.ndarray
        Diagonal matrix of singular values.
    Vt : np.ndarray
        Right singular vectors (V transpose) of the input matrix.
    condition_number : float
        Condition number of the matrix, calculated as the ratio of the largest to smallest singular values.
    matrix_inverse : np.ndarray
        Pseudo-inverse of the input matrix.

    Raises:
    ------
    ValueError:
        If the matrix is singular and does not have an inverse.
    """

    # Step 1: Compute A^T * A, which is used to determine the right singular vectors
    AtA = np.dot(matrix.T, matrix)
    
    # Step 2: Eigen decomposition for right singular vectors (V) and corresponding singular values
    eigvals_AtA, V = np.linalg.eig(AtA)
    
    # Step 3: Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigvals_AtA)[::-1]
    eigvals_AtA = eigvals_AtA[sorted_indices]
    V = V[:, sorted_indices]
    
    # Step 4: Calculate singular values (square root of eigenvalues of A^T * A)
    singular_values = np.sqrt(eigvals_AtA)
    if singular_values[-1] == 0:
        raise ValueError("Matrix is singular and does not have an inverse.")
    
    # Step 5: Calculate U (left singular vectors) using A * V / singular_values
    U = matrix @ V / singular_values
    U /= np.linalg.norm(U, axis=0)  # Normalize U

    # Step 6: Construct the diagonal matrix S with singular values
    S_diag = np.diag(singular_values)
    
    # Step 7: Calculate the pseudo-inverse of S (S_inv) for non-zero singular values
    S_inv_diag = np.diag(1 / singular_values)
    
    # Step 8: Calculate the pseudo-inverse of the matrix
    matrix_inverse = V @ S_inv_diag @ U.T

    # Step 9: Calculate the condition number as the ratio of the largest to the smallest singular value
    condition_number = singular_values[0] / singular_values[-1]

    return U, S_diag, V.T, condition_number, matrix_inverse

# Example test matrix
matrix = np.array([[1, 7,6], [4, 19,22],[13,12,201]], dtype=float)

# Run the custom SVD function
try:
    U_custom, S_diag_custom, Vt_custom, condition_number_custom, matrix_inverse_custom = custom_svd(matrix)

    # Output all components of the SVD
    print("Custom SVD U:\n", U_custom)
    print("Custom SVD S (Diagonal):\n", S_diag_custom)
    print("Custom SVD V^T:\n", Vt_custom)
    print("Custom Condition Number:", condition_number_custom)
    print("Custom Matrix Inverse:\n", matrix_inverse_custom)

    # Matrix reconstruction using the custom SVD components
    reconstructed_matrix_custom = U_custom @ S_diag_custom @ Vt_custom
    print("Custom SVD Matrix Reconstruction:\n", reconstructed_matrix_custom)

except ValueError as e:
    print(e)

print("_________________________")

# Using NumPy's SVD for comparison
U_np, S_np, Vt_np = np.linalg.svd(matrix)
S_diag_np = np.diag(S_np)
matrix_inverse_np = np.dot(Vt_np.T, np.dot(np.diag(1 / S_np), U_np.T))
condition_number_np = np.linalg.cond(matrix)

# Output all components from NumPy's SVD
print("\nNumPy U:\n", U_np)
print("NumPy S (Diagonal):\n", S_diag_np)
print("NumPy V^T:\n", Vt_np)
print("NumPy Matrix Inverse:\n", matrix_inverse_np)
print("NumPy Condition Number:\n", condition_number_np)

# Matrix reconstruction using NumPy SVD components
reconstructed_matrix_np = U_np @ S_diag_np @ Vt_np
print("NumPy SVD Matrix Reconstruction:\n", reconstructed_matrix_np)


Custom SVD U:
 [[ 0.03213868 -0.34801801 -0.9369368 ]
 [ 0.11563616 -0.92983126  0.34934525]
 [ 0.99277157  0.11957127 -0.01035992]]
Custom SVD S (Diagonal):
 [[203.23351518   0.           0.        ]
 [  0.          18.89523203   0.        ]
 [  0.           0.           0.32941469]]
Custom SVD V^T:
 [[ 0.06593752  0.07053618  0.99532753]
 [-0.1329921  -0.98797754  0.07882564]
 [ 0.9889213  -0.13756827 -0.05576403]]
Custom Condition Number: 616.9533995119826
Custom Matrix Inverse:
 [[-2.81027668e+00  1.05533597e+00 -3.16205534e-02]
 [ 4.09486166e-01 -9.72332016e-02 -1.58102767e-03]
 [ 1.57312253e-01 -6.24505929e-02  7.11462451e-03]]
Custom SVD Matrix Reconstruction:
 [[  1.   7.   6.]
 [  4.  19.  22.]
 [ 13.  12. 201.]]
_________________________

NumPy U:
 [[-0.03213868 -0.34801801 -0.9369368 ]
 [-0.11563616 -0.92983126  0.34934525]
 [-0.99277157  0.11957127 -0.01035992]]
NumPy S (Diagonal):
 [[203.23351518   0.           0.        ]
 [  0.          18.89523203   0.        ]
 [  0.  